# 1. Scrapping Part

## scraping pdf :

1. scraping the 'company's id' who have publish something in the "moniteur" in a given periode.
    2 output :  list of id's 
    
    [
    "0691625539",
    "0702777668",
    "0702795187",
    "0702775886",
    "0702799543",
    "0702768562",
    "0702761832",
    "0701797176",
    "0701791931",
    "0702764307",
    "0701819843",
    "0702798157",
    
                list of url 
                
    [
    "https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer=0687513729",
    "https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer=0687518479",
    "https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer=0687490072",
    "https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer=0686796523",
    "https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer=0687502742",

In [2]:
import datetime

def dategenerator():
    dt = datetime.datetime(2018, 1, 1)
    end = datetime.datetime(2018, 12, 31)
    step = datetime.timedelta(days=1)

    result = []

    while dt < end:
        result.append(dt.strftime('%Y-%m-%d'))
        dt += step
    return result 

In [4]:
import scrapy
import json
import re
#from utils.date import dategenerator

# launch script : scrapy runspider THISFILE.py 

class BlogSpider(scrapy.Spider):
    name = 'DE STAATSBLADMONITOR 2018 '

    start_urls = []

    daterange = dategenerator()
    for date in daterange:
        start_urls.append('https://www.staatsbladmonitor.be/oprichtingen-bedrijven.html?datum='+date)


    data_final = []
    token = []

    def parse(self, response):
        for data in response.css('.data::text'):
            if re.match('^[0-9]+$', data.get()):
                self.data_final.append('https://www.staatsbladmonitor.be/bedrijfsfiche.html?ondernemingsnummer='+data.get())
                self.token.append(data.get())

        with open('link_01.json', 'w', encoding='utf-8') as f:
            json.dump(self.data_final, f, ensure_ascii=False, indent=4)

        with open('token.json', 'w', encoding='utf-8') as g:
            json.dump(self.token, g, ensure_ascii=False, indent=4)  

2.from the url's scraped in the first step , we scrap the link to the pdf from 'le moniteur'

    output example :
    
    {"http://www.ejustice.just.fgov.be/tsv_pdf/2018/01/08/18300807.pdf": {"VAT": "0687513729", "source_ref": "18300807", "source_date": "20180108"}

In [5]:
import pandas as pd
import json
from fake_useragent import UserAgent 
import requests
from bs4 import BeautifulSoup
import re
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from requests.exceptions import SSLError
from OpenSSL.SSL import SysCallError
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


urls = pd.read_json('link_01_2018.json')

# 1000 first for testing
urls = urls[0][:1000].tolist()

def link_pdf():
    ua = UserAgent()
    urls_pdf = {}
    for i in urls:
        try:
            response = requests.get(i,  headers={'User-Agent': ua.random}, verify=False)
            print(response)
            soup = BeautifulSoup(response.text, 'html.parser')

            for link in soup.findAll('a', attrs={'href': re.compile("pdf$")}):
                links = link.get('href')
                #urls_pdf.append([link.get('href'), i[-10:]])
                urls_pdf[link.get('href')] = {'VAT': i[-10:],
                                              'source_ref': links[-12:-4],
                                              'source_date': str(links[-23:-19])+str(links[-18:-16])+str(links[-15:-13])
                                              }

        except SysCallError :
            print(f"SysCallError for {i[-10]}")


    return urls_pdf

result = link_pdf()

with open('link_pdf_2018.json', 'w') as outfile:
    json.dump(result, outfile)  

ValueError: Expected object or value

3. downloading pdf , name pdf = yyyyMMDD_VATNUM.pdf        

In [ ]:
import os.path
import time
import requests
from fake_useragent import UserAgent
import sys
import json



print('Beginning file download with wget module')

#urlstest = "http://www.ejustice.just.fgov.be/tsv_pdf/2019/01/03/19300336.pdf"

with open('link_pdf_2018.json', 'r') as f:
        urls = json.load(f)

ua = UserAgent()

for key in urls :
    try:
        time.sleep(0.1)
        print(f'trying: {urls[key]}')
        pdfname = str(urls[key]['source_date'])+'_'+str(urls[key]['VAT'])
        #print(pdfname)
        
        if os.path.exists('./pdf/'+str(pdfname)+'.pdf') is False:

            myfile = requests.get(key, headers={'User-Agent': ua.random})
            print(f'yo : {key}')
            with open(f'pdf/{pdfname}.pdf', 'wb') as file:
                file.write(myfile.content)
        else:
            pass

    except:
        print(f'ERROR url: {key}')
        print(sys.exc_info())

# 2. Text extraction:

Then, 
- we extract the pdf content:

    - we choose to convert the pdf's to images :
        ADD IMAGE HERE
    - crop the iamges 

    - and extract the text by OCR using tesseract and pytesseract


- determine the language :

 with LanguageDetector from spacy_langdetect library
 
 
- scrap the object of the document from 'le moniteur' 
 
- fill a mongodb  with :

    _id : VAT
    
    documents :{YYYYMMDD: {'text' : EXTRACTED TEXT , 'object': SCRAPED OBJECT }}
    
    language : extracted language
    
    
NB : text and cropped images are save in the filesystem    

In [ ]:
import os
import glob
import spacy
from pymongo.errors import DuplicateKeyError
from spacy_langdetect import LanguageDetector
from pymongo import MongoClient
from wand.image import Image as WandImg
from PIL import Image
import pytesseract
from scrap_act_object_2 import act_obj

pytesseract.pytesseract.tesseract_cmd = r'C:\Users\sebch\AppData\Local\Tesseract-OCR\tesseract.exe'
#SEB path  r'C:\Users\sebch\AppData\Local\Tesseract-OCR\tesseract.exe'
#original path  r'C:\Program Files\Tesseract-OCR\tesseract.exe'

### German, nederlands and frech Tesseract packages can be download from  : https://github.com/tesseract-ocr/tesseract/wiki/Data-Files

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)

def pdfsToTxt(folderPath):
    files = []

    for r, d, f in os.walk(folderPath):
        for file in f:
            if ".pdf" in file:
                files.append(os.path.join(r, file))

    for f in files:
        existing_txts = glob.glob(format_txt_filename(f, "dummy").replace("dummy", "*"))
        if existing_txts :
            print("TXT file already exists, skipping text detection and db insertion")
        else:
            #extract text from images
            image_path_list = convert_to_images(f, 300)
            total_page_count = str(len(image_path_list))
            page_count = 0
            pdf_string = ""
            for image_path in image_path_list :
                page_count += 1
                pdf_img = Image.open(image_path)
                pdf_string += pytesseract.image_to_string(pdf_img, lang='fra+deu+nld')
                print("Processed page " + str(page_count) + "/" + total_page_count
                      + " (" + image_path + ")")

            #detect language
            doc = nlp(pdf_string)
            language = doc._.language["language"]

            #write txt
            txt_file_path = format_txt_filename(f, language)
            txt_file = open(txt_file_path, "w+", encoding='utf-8')
            txt_file.write(pdf_string)
            print("Txt created: " + txt_file_path)
            txt_file.close()

            #write to db
            vat_nr_date = os.path.basename(f).replace(".pdf", "")
            uid = vat_nr_date[-10:]
            date = vat_nr_date[-19:-11]
            statute_dict = {"_id" : uid,
                            "documents": {f'{date}': {'text' : pdf_string, 'object': act_obj(uid[1:], date)}},
                            "language" : language}
            client = MongoClient()
            db = client.kpmg
            stat_coll = db.statutes
            stat_doc = stat_coll.find_one({"_id": uid})
            if stat_doc:
                stat_coll.update_one({'_id': uid},
                                      {'$set': {f"documents.{date}": {'text': pdf_string, 'object': act_obj(uid[1:], date)}}},
                                      upsert=True
                                      )
            else:
                insert_result = stat_coll.insert_one(statute_dict)
                if insert_result.acknowledged:
                    print("Document inserted into db kmpg, collection statutes, with _id " + str(
                        insert_result.inserted_id))

def encode_newlines(text) :
    return text.replace("\\r\\n", "\r\n").replace("\\n", "\n").replace("\\r", "\r")

def remove_newlines(text) :
    return text.replace("\\r\\n", " ").replace("\\n", " ").replace("\\r", " ")

def format_txt_filename(original_pdf, language) :
    return os.path.abspath(original_pdf).replace(".pdf", "_").replace("\\pdf", "\\txt") + \
    language + \
    ".txt"

def convert_to_images(file, resolution) :
    file_path = os.path.abspath(file)
    img_path_template = file_path.replace("\\pdf", "\\png").replace(".pdf", "")

    image_paths = glob.glob(img_path_template + "*")
    if image_paths :
        print("Images for " + file_path + " already exist, skipping image conversion")
    else :
        with WandImg(filename=file_path, resolution=resolution) as source:
            source.compression_quality = 99
            images = source.sequence
            nr_of_pages = len(images)
            image_paths = []
            for i in range(nr_of_pages):
                img_path = img_path_template + "_" + str(i) + ".png"
                #croping pages (if first pages else the others)
                if i == 0:
                    images[i].crop(int(images[i].size[0] * 0.16), int(images[i].size[1] * 0.20), int(images[i].size[0] * 0.95),
                                int(images[i].size[1] * 0.92))
                else:
                    images[i].crop(int(images[i].size[0] * 0.16), int(images[i].size[1] * 0.052), int(images[i].size[0] * 0.95),
                                int(images[i].size[1] * 0.92))
                WandImg(images[i]).save(filename=img_path)
                image_paths.append(img_path)
            print("Image(s) created: " + str(image_paths))

    return image_paths


pdfsToTxt(".\\pdf")


### scraping of the text object

In [ ]:
from bs4 import BeautifulSoup as bs
import requests


def act_obj(uid, date):
    date = str(date)
    date_f = str(date)[:4]+'-'+str(date)[4:6]+'-'+str(date[6:8])

    url = f"https://www.ejustice.just.fgov.be/cgi_tsv/tsv_l_1.pl?lang=fr&sql=btw+contains+%27{uid}%27&fromtab=TSV&rech=1&pdda=&pddm=&pddj=&pdfa=&pdfm=&pdfj=&naam=&postkode=&localite=&numpu=&hrc=&akte=&btw={uid}&jvorm=&land=&set2=&set3="

    response = requests.get(url)
    soup = bs(response.text,features="lxml").find_all('td')
    for i in range(0, round(len(soup) / 2) - 1):
        test02 = soup[-2 + (-3 * i)].find_all('br')
        test03 = test02[-1].next_sibling
        if date_f in test03:
            result = test02[-1].previous_sibling
            return result

#print(act_obj(687501257, 20180108))

# 3. Let's then take all the metadata we can get from the governement website and fill the db with.


        for this we use a reusable set of csv (https://economie.fgov.be/fr/themes/entreprises/banque-carrefour-des/services-pour-tous/banque-carrefour-des-2 ) update every month.
        
        rem : we use pandas to recover the data , for sure some better way are possible.

In [ ]:
## need the csv and the token list generate in the first step

from pymongo import MongoClient
import pandas as pd
import json
from dictionaries import load_dict
from meta_gen import dict_gen

print('loading tables')

activity = pd.read_csv('./OpenData/activity.csv')
table02 = pd.read_csv('./OpenData/address.csv')
table03 = pd.read_csv('./OpenData/code.csv')
table04 = pd.read_csv('./OpenData/contact.csv')
table05 = pd.read_csv('./OpenData/denomination.csv')
table06 = pd.read_csv('./OpenData/enterprise.csv')
table07 = pd.read_csv('./OpenData/establishment.csv')


print('tables loaded')
#need probably to be re-scraped to match with the pdf's
vat_numbers = pd.read_json('token_test.json')

#TODO for testing purpose - remove
#vat_numbers = vat_numbers[:50]

vat_numbers = "0" + vat_numbers.astype(str)
##formatting TVA number not needed here anymore
#vat_numbers[1]= vat_numbers[0].str[:4] + '.' + vat_numbers[0].str[4:7] + \
#                '.'+vat_numbers[0].str[7:10]
vat_uids = vat_numbers[0].tolist()
#vat_formatted = vat_numbers[1].tolist()

print(vat_uids)
#print(vat_formatted)


print('loading dictionaries')
ActivityGroup_dic,  JuridicalForm_dic, JuridicalSituation_dic, Nace2003_dic, Nace2008_dic,TypeOfEnterprise_dic = load_dict(table03)


def update_statutes_in_db(uids):
    client = MongoClient()
    db = client.kpmg
    stat_coll = db.statutes
    for uid in uids :
        stat_doc = stat_coll.find_one({"_id": uid})
        if stat_doc :
            meta_dic = dict_gen(uid, activity, table02, table04, table05, table06, table07,
                ActivityGroup_dic,  JuridicalForm_dic, JuridicalSituation_dic, Nace2003_dic, Nace2008_dic, TypeOfEnterprise_dic )
            meta_dic.update(stat_doc) #this way fields in stat_doc do not get overwritten
            stat_coll.update_one({'_id': uid}, {"$set": meta_dic}, upsert=False)
            print("Found statute with uid " + uid + ". Updated meta data where empty")

        else :
            print("Document with uid " + str(uid) + " not found in database")


update_statutes_in_db(vat_uids)

# this part generate the meta data for one given VAT NUMBER
#very slow ...

import pandas as pd

#from scrap_act_object import act_obj
def dict_gen(uid, activity, table02, table04, table05, table06, table07,
             ActivityGroup_dic, JuridicalForm_dic, JuridicalSituation_dic, Nace2003_dic, Nace2008_dic, TypeOfEnterprise_dic):

    meta_dic = dict()
    uid_f = str(uid[:4]+'.'+str(uid[4:7])+'.'+str(uid[7:10]))
    uid = int(uid)
    # formatting 2 features from table 06 to match with the code table (table03)

    temp = table06.JuridicalForm.fillna(0).astype(int).astype(str)
    temp2 = table06.JuridicalSituation.fillna(0).astype(int).astype(str)
    temp = ["0" + i if len(i) == 2 else "00" + i if len(i) == 1 else i for i in temp]
    temp2 = ["0" + i if len(i) == 2 else "00" + i if len(i) == 1 else i for i in temp2]
    table06.JuridicalForm = temp
    table06.JuridicalSituation = temp2

    table06_row = table06[table06['EnterpriseNumber'] == uid_f]
    table05_row = table05[table05['EntityNumber'] == uid_f]
    table05_row_abbr = table05[(table05['EntityNumber'] == uid_f) & (table05.TypeOfDenomination == 2)]
    table01_row_main = activity[(activity['EntityNumber'] == uid_f) & (activity.Classification == 'MAIN')].astype(str)
    table01_row_sec = activity[(activity['EntityNumber'] == uid_f) & (activity.Classification == 'SECO')].astype(str)
    table07_row = table07[table07['EnterpriseNumber'] == uid_f]
    table02_row = table02[table02['EntityNumber'] == uid_f]
    table04_row_mail = table04[(table04['EntityNumber'] == uid_f) & (table04['ContactType'] == 'EMAIL')]
    table04_row_phone = table04[(table04['EntityNumber'] == uid_f) & (table04['ContactType'] == 'TEL')]

    if len(table06_row) == 0:
        print("No data found for vat number " + uid_f + " in the CSVs")
    else:
        meta_dic['VAT Number'] = table06_row.iloc[0]['EnterpriseNumber']

        #meta_dic['act object'] = act_obj(uid)

        meta_dic['Denomination'] = table05_row.iloc[0]['Denomination']

        meta_dic['Abbr'] = (lambda x: table05_row_abbr.iloc[0]['Denomination']
        if len(table05_row_abbr) != 0 else 'NONE')(uid_f)

        meta_dic['ActivityGroup'] = (lambda x: table01_row_main.iloc[0]['ActivityGroup']
        if len(table01_row_main) != 0 else "none")(uid_f)

        meta_dic['Main activity'] = (
            lambda x: [table01_row_main.iloc[0]['NaceVersion'], table01_row_main['NaceCode'].unique().tolist()]
            if len(table01_row_main) != 0 else "none")(uid_f)

        meta_dic['Secondary activity'] = (lambda x: table01_row_sec['NaceCode'].unique().tolist()
        if len(table01_row_sec) != 0 else "none")(uid_f)

        meta_dic['Foundation Date'] = table06_row['StartDate'].iloc[0]

        meta_dic['Establishment #'] = len(table07_row)

        meta_dic['Establishment StartDate'] = table07_row['StartDate'].to_list()

        meta_dic['JuridicalSituation'] = str(table06_row.iloc[0]['JuridicalSituation'])

        meta_dic['TypeOfEnterprise'] = str(table06_row.iloc[0]['TypeOfEnterprise'])

        meta_dic['JuridicalForm'] = str(table06_row.iloc[0]['JuridicalForm'])

        # meta_dic['Language']= table06_row.iloc[0]['TypeOfEnterprise']

        meta_dic['Zipcode'] = table02_row.iloc[0]['Zipcode']

        meta_dic['Street'] = table02_row.iloc[0]['StreetFR']

        meta_dic['HouseNumber'] = table02_row.iloc[0]['HouseNumber']

        meta_dic['Mail'] = (lambda x: table04_row_mail.iloc[0]['Value']
        if len(table04_row_mail) != 0 else 'NA')(uid_f)

        meta_dic['Phone'] = (lambda x: table04_row_phone.iloc[0]['Value']
        if len(table04_row_phone) != 0 else 'NA')(uid_f)
        
        
        # "translation of the codes present in the csv"
        

        for KEY, VALUE in ActivityGroup_dic.items():
            if meta_dic['ActivityGroup'] == KEY:
                meta_dic['ActivityGroup'] = VALUE

        for KEY, VALUE in JuridicalForm_dic.items():
            if meta_dic['JuridicalForm'] == KEY:
                meta_dic['JuridicalForm'] = VALUE

        for KEY, VALUE in JuridicalSituation_dic.items():
            if meta_dic['JuridicalSituation'] == KEY:
                meta_dic['JuridicalSituation'] = VALUE

        for KEY, VALUE in TypeOfEnterprise_dic.items():
            if meta_dic['TypeOfEnterprise'] == KEY:
                meta_dic['TypeOfEnterprise'] = VALUE

        if meta_dic['Main activity'][0] == 2003:
            for KEY, VALUE in Nace2003_dic.items():
                for i in range(0, len(meta_dic['Main activity'])):
                    if meta_dic['Main activity'][1][i] == KEY:
                        meta_dic['Main activity'][1][i] = (KEY, VALUE)

                for i in range(0, len(meta_dic['Secondary activity'])):
                    if meta_dic['Secondary activity'][i] == KEY:
                        meta_dic['Secondary activity'][i] = (KEY, VALUE)
        else:
            for KEY, VALUE in Nace2008_dic.items():
                for i in range(0, len(meta_dic['Main activity'][1])):
                    if meta_dic['Main activity'][1][i] == KEY:
                        meta_dic['Main activity'][1][i] = (KEY, VALUE)

                for i in range(0, len(meta_dic['Secondary activity'])):
                    if meta_dic['Secondary activity'][i] == KEY:
                        meta_dic['Secondary activity'][i] = (KEY, VALUE)

        #print(meta_dic)

    return(meta_dic)



#we need this to "translate" code in readable information

def load_dict(table03):
    trad = table03[['Category', 'Code', 'Description']][table03.Language == 'FR']
    dict_list = []
    for cat in trad.Category.unique():
        temp_trad = trad[trad.Category == cat]
        keys = temp_trad['Code'].to_list()
        values = temp_trad['Description'].to_list()
        dict_list.append(dict(zip(keys, values)))

    ActivityGroup_dic = dict_list[0]
    # Classification_dic = dict_list[1]
    # ContactType_dic = dict_list[2]
    # EntityContact_dic = dict_list[3]
    JuridicalForm_dic = dict_list[4]
    JuridicalSituation_dic = dict_list[5]
    # Language_dic = dict_list[6]
    Nace2003_dic = dict_list[7]
    Nace2008_dic = dict_list[8]
    #Status_dic = dict_list[9]
    # TypeOfAddress_dic = dict_list[10]
    # TypeOfDenomination_dic = dict_list[11]
    TypeOfEnterprise_dic = dict_list[12]
    return ActivityGroup_dic,  JuridicalForm_dic, JuridicalSituation_dic,\
           Nace2003_dic, Nace2008_dic, TypeOfEnterprise_dic

## 4. we manage to be able to split the text into articles 


in the condition le script can find some key word in the text


we integrate the traduction in this part too even if it make the code less readable.

In [ ]:
from pymongo import MongoClient
import json
from googletrans import Translator
from json.decoder import JSONDecodeError

translator = Translator()

client = MongoClient()
db = client.kpmg
stat_coll = db.statutes

#key word to be found in the text:
key_word = ["Art", "Article", 'Art.', 'Art,', 'Ast', 'Ari', "art", "At", "ARTICLE", "ART", 'Ant', 'Artikel', 'ARTIKEL', 'article', 'Articdle']
#VAT number list
filename = r'C:\Users\sebch\Desktop\kpmg\GitHub Repo\KPMG_project\scraping_final\scraping_staatblad_seb\token_test.json'
with open(filename, 'r') as f:
    uids = json.load(f)

#uids= uids[:10]
log = []


for uid in uids:
    try:
        # fetching cie first document data
        print(f'trying {uid}')
        stat_doc = stat_coll.find_one({"_id": uid})
        doc_data = stat_doc['documents']
        date = [i for i in doc_data][0]

        #checking if it is a consitution doc
        obje = doc_data[date]['object']
        constitution = ['CONSTITUTION', 'OPRICHTING', 'STATUTS', 'STATUTEN']
        for obj in obje.split():
            if obj in constitution:

                #look for key_word in text splitted
                text = next(iter(next(iter(doc_data.values())).values()))
                
                #translate the full text and add to the db
                
                try:
                    translation = translator.translate(text, dest='en')
                    stat_coll.update_one({'_id': uid},
                                         {'$set':
                                              {f"documents.{date}.translation": translation.text}
                                         },
                                         upsert=True
                                         )
                    print("traduction added")
                except JSONDecodeError:
                    print('unable to translate text')
                    
                    
                    
                # split the text by "lignes"
                split = text.split('\n')
                #index of each ligne containing one the key_word
                cut_index = []
                for group in split:
                    #removing useless characters and splitting by 'words'
                    for word in group.translate({ord(i): ' ' for i in ":[‘'(),. "}).split():
                        if word in key_word:
                            cut_index.append(split.index(group))
                            
                            
                # text until first occurence of on e key_word
                start = split[:cut_index[0] - 1]
                #between two occurences
                articles = []
                for i in cut_index:
                    if i != cut_index[-1]:
                        articles.append(split[i:cut_index[cut_index.index(i) + 1]])
                #after the last occurences
                end = split[cut_index[-1] + 1:]

                sections = [start] + articles + [end]
                #avoiding list of several string (need to be fine tuned)
                for element in sections:
                    temp = str()
                    for el in element:
                        temp += el

                    sections[sections.index(element)] = temp
                    
                #injection in the db    
                stat_coll.update_one({'_id': uid},
                                     {'$addToSet': {
                                         f"documents.{date}.sections": {str(sections.index(element)): element for element in
                                                                        sections}
                                     }},
                                     upsert=True
                                     )
                #translation articles by articles
                try:
                    sections_t = [translator.translate(i, dest='en').text for i in sections]
                    print(sections_t)
                    stat_coll.update_one({'_id': uid},
                                         {'$addToSet': {
                                             f"documents.{date}.sections": {str(sections_t.index(element)): element for
                                                                            element in
                                                                            sections_t}
                                         }},
                                         upsert=True
                                         )
                except JSONDecodeError :
                    print('unable to translate articles')

                log.append(uid)
                print(f'added {uid}')
            else :
                pass
    except TypeError:
        print(f'no document found for {uid}')
    except IndexError:
        print(f'cannot parse document for {uid}')
    except AttributeError:
        print(f'cannot find document object for {uid}')

with open('log.json', 'w', encoding='utf-8') as g:
    json.dump(log, g, ensure_ascii=False, indent=4)